In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd 
import os
import pickle
import time 


from ipynb.fs.full.utils import calculate_box, draw_box, get_landmarks_snapshot, get_landmarks_angles, draw_finger_angles

PATH = 'E:\RealTimeObjectDetection'
DATA_PATH = PATH + '\data'

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
classifier = pickle.load(open('model.sav', 'rb'))

https://google.github.io/mediapipe/images/mobile/hand_landmarks.png

In [4]:
joint_list = [[0,1,2],[1,2,3],[2,3,4],[0,5,6],[5,6,7],[6,7,8],[0,9,10],[9,10,11],[10,11,12],[0,13,14],[13,14,15],[14,15,16],[0,17,18],[17,18,19],[18,19,20]]

#### import labels

In [5]:
import json

with open("labels.txt") as f:
    labels = json.load(f)

In [6]:
#lock 
def sequence_detection(y_pred, latch, sequence): 
    if len(latch) < 3:
        latch.append(y_pred)
    else:
        if latch[0] == latch[1] and latch[1] == latch[2]:
            if len(sequence) == 0:
                sequence.append(latch[0])
            elif sequence[-1] != latch[2]:
                sequence.append(latch[0])
        latch[0] = latch[1]
        latch[1] = latch[2]
        latch[2] = y_pred

    return sequence


In [7]:
def check_lock(lock, unlock):
    for index in range(len(unlock)-len(lock)+1):
        unlock_ = []
        for i, _ in enumerate(lock):
            unlock_.append(unlock[index+i])
        if unlock_ == lock:
            return False
    return True
    

In [20]:
sequence_mode_on = False
check_lock_mode_on = False
mode_box = [400, 340, 600, 140]
start_ = True
start = 0
locked = True
sequence = []
sequence_ = []
latch = []

cap = cv2.VideoCapture(0)
landmarksData = pd.DataFrame()

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=1) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Wrap open-cv with mediapipe
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image = cv2.flip(image,1)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.rectangle(image,(mode_box[0],mode_box[1]),(mode_box[2],mode_box[3]),(0,255,0),2)
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                #draw box
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                         mp_drawing.DrawingSpec(color=(245, 135, 66), thickness=2, circle_radius=2),
                                         mp_drawing.DrawingSpec(color=(171, 66,0 ), thickness=2, circle_radius=2))
                box = calculate_box(image,results.multi_hand_landmarks)
                image = draw_box(True, image,box)
                #draw angles
#                 image = draw_finger_angles(image, results, joint_list)
                #predict
                y_pred = classifier.predict(get_landmarks_angles(results.multi_hand_landmarks, joint_list).reshape(1,-1))
                # start authentication
                if (box[0]>mode_box[0] and box[1]<mode_box[1] and box[2]<mode_box[2] and box[3]>mode_box[3] and y_pred[0]== 0):
                    check_lock_mode_on = !check_lock_mode_on
                    locked = False
                # set lock 
                if (box[0]>mode_box[0] and box[1]<mode_box[1] and box[2]<mode_box[2] and box[3]>mode_box[3] and y_pred[0]== 1):
                    lock = []
                    sequence= []
                    sequence_mode_on = True
                if start_:
                    start = time.time()
                    print(start)
                    start_ = False
                    stop = time.time()
                if (stop - start >= 5 and sequence_mode_on):
                    print("end")
                    sequence_mode_on = False
                    print(lock)
                stop = time.time()
                #set sequence
                if sequence_mode_on:
                    lock = sequence_detection(y_pred[0], latch, sequence)[1:]
                #check lock
                if check_lock_mode_on:
                    unlock = sequence_detection(y_pred[0], latch, sequence_)
                    if len(unlock) >= len(lock):
                        locked = check_lock(lock, unlock)
                        if locked == False: 
                            print("unlocked")
                            check_lock_mode_on = False
                #draw label
                for label in labels:
                    if y_pred == label['id']:
                        cv2.putText(image, label['label'], (box[0] + 5, box[1] - 4),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.imshow('Hand tracking', image)
        
        # Onkey functions
        k = cv2.waitKey(33)
        if k==97:    # Get snapshot when pressed "a"
            if results.multi_hand_landmarks:
                data = get_landmarks_snapshot(results.multi_hand_landmarks)
                landmarksData = pd.concat([landmarksData, data], ignore_index=True)
        elif k==115: # Save data when pressed "s"
            landmarksData.to_csv(r'E:\RealTimeObjectDetection\data\data.csv', index=False)
        elif k==49: # Start sequence when pressed "1"
            print("start")
            lock = []
            sequence= []
            sequence_mode_on = True
        elif k==50: # End sequence when pressed "2"
            print("end")
            sequence_mode_on = False
            print(lock)
        elif k==51: # Check lock sequence when pressed "3"
            check_lock_mode_on = !check_lock_mode_on
            locked = False
        elif k==113: # Close window when pressed "q"
            break
            
cap.release()
cv2.destroyAllWindows()

1653917394.0363753
end
[0.0, 1.0, 0.0, 1.0, 0.0, 3.0, 0.0]
unlocked
